# **📊 DSCBI Macroeconomic Indicators Visualization Project**

## **Description:**
* The DSCBI Visualization Project aims to create an interactive and dynamic dashboard using Streamlit and Python to visualize macroeconomic indicators produced by the department.
* It will provide stakeholders with quick insights into sectoral performance, indicator trends, and projections for planning and policy analysis.
## **🎯 Objectives**
* Centralize visualization of macroeconomic indicators (historical and projected).
* Enable users to filter and view data by sector, indicator, and year range.
* Provide key metric summaries (average, max, and long-term projected values).
* Present interactive charts for both calendar year and fiscal year data.
* Offer a downloadable table view for selected indicators.
* Highlight data sources, stakeholders, and important reference links.

## **🧱 Project Structure**


dscbi_visualization/
│
├── data/
│   ├── macro_indicators.csv
│   └── metadata_sources.csv
│
├── app/
│   ├── __init__.py
│   ├── main.py                # Main Streamlit entry point
│   ├── utils.py               # Helper functions for loading data, computing metrics
│   ├── charts.py              # Chart generation functions
│   ├── layout.py              # Layout and style configurations
│
├── assets/
│   ├── logo.png
│   └── style.css
│
├── requirements.txt
├── README.md
└── .gitignore


## **🧭 Dashboard Layout**

| Section                                   | Description                                                                                  |
| ----------------------------------------- | -------------------------------------------------------------------------------------------- |
| **1. Sidebar (Selection Panel)**          | Contains filters for: <br>– Sector Selection <br>– Indicator Selection <br>– Years Selection |
| **2. Title Header**                       | Attractive title banner with department logo, project title, and tagline.                    |
| **3. Main Body**                          | Divided into three key parts:                                                                |
|     • **Key Metrics Panel**               | Displays summary indicators (Max value, Average value, Long-term projection).                |
|     • **Charts Section**                  | Two charts: <br>– Calendar Year Trends <br>– Fiscal Year Trends                              |
|     • **Outputs Table**                   | Interactive table displaying data based on selected filters (with historical + projections). |
| **4. Key Sources & Stakeholders Section** | Displays key institutions, data providers, and important links for reference.                |


### User Interaction

* User selects Sector, Indicator(s), and Year Range from the sidebar.

### Data Filtering

* The app dynamically filters the dataset based on selected parameters.

### Computation

* Metrics (Average, Max, Long-term projection) are calculated.

### Visualization

#### The filtered data is plotted into two main charts:

* Calendar Year Chart

* Fiscal Year Chart

### Table Generation

* A dynamic table displays the corresponding data.

### Reference Section

* Displays key sources and links at the bottom.

In [1]:
# ==============================
# 📦 SECTION 1: IMPORT LIBRARIES
# Load required Python libraries for Streamlit app, data handling, and visualization.
# ==============================
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from io import StringIO
import re
from pathlib import Path

# display columns and rows settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# ==============================




In [ ]:





import streamlit as st

# ======================= PAGE CONFIGURATION =======================
st.set_page_config(page_title="Dashboard Example", layout="wide")

# ======================= CUSTOM CSS =======================
# def load_css():
#     return """
#     <style>
#     /* ===== GLOBAL ===== */
#     .stApp {
#         background-color: #021a36;
#         color: #e6eaf0;
#         font-family: "Inter", sans-serif;
#     }

#     /* ===== SIDEBAR ===== */
#     [data-testid="stSidebar"] {
#         background-color: #0b1e3b;
#         color: #e6eaf0;
#         padding: 20px;
#     }

#     /* ===== METRIC CARDS ===== */
#     .metric-card {
#         background-color: #03295c;
#         color: #e6eaf0;
#         padding: 20px;
#         border-radius: 15px;
#         text-align: center;
#         box-shadow: 2px 2px 10px rgba(0,0,0,0.3);
#         margin-bottom: 20px;
#         transition: transform 0.2s;
#     }
#     .metric-card:hover {
#         transform: scale(1.05);
#     }
#     .metric-value {
#         font-size: 28px;
#         font-weight: bold;
#         margin-top: 10px;
#     }
#     .metric-label {
#         font-size: 16px;
#         opacity: 0.8;
#     }
#     </style>
#     """


st.markdown(
    """
    <div style="background-color:white; padding:20px;">
        <header style="font-size:24px; color:darkblue; font-weight:bold;">
            My Header Title
        </header>
        <div>
        <h1>
        MACRO DASHBOARD is GREAT
        </h1>
        </div>
    </div>
    """,
    unsafe_allow_html=True
)


#st.markdown(load_css(), unsafe_allow_html=True)



# ======================= SIDEBAR =======================
st.sidebar.title("Dashboard Controls")
selected_month = st.sidebar.selectbox("Select Month", ["January", "February", "March", "April"])
st.sidebar.slider("Filter Revenue (k$)", min_value=0, max_value=100, value=(10, 50))

# ======================= HEADER =======================

st.markdown("<h1 style= 'text-align:center; font-family:verdana; color:white; font-size:200%; background-color:darkblue'>" \
"ANDREW MUSHOKAMBERE</h1>", unsafe_allow_html=True)
st.markdown("<h1 style='text-align:center; color:#e6eaf0;'>Company Dashboard</h1>", unsafe_allow_html=True)

# ======================= METRICS =======================
# Create columns for cards
col1, col2, col3, col4 = st.columns(4)

# Define card HTML template
def create_metric_card(label, value):
    card_html = f"""
    <div class="metric-card">
        <div class="metric-label">{label}</div>
        <div class="metric-value">{value}</div>
    </div>
    """
    return card_html

# Example KPIs
with col1:
    st.markdown(create_metric_card("Revenue", "$25,000"), unsafe_allow_html=True)
with col2:
    st.markdown(create_metric_card("Profit", "$7,500"), unsafe_allow_html=True)
with col3:
    st.markdown(create_metric_card("Customers", "1,200"), unsafe_allow_html=True)
with col4:
    st.markdown(create_metric_card("Orders", "350"), unsafe_allow_html=True)

# ======================= SECOND ROW OF METRICS =======================
col1, col2  = st.columns(2)
with col1:
    st.button("Button1")
with col2:
    st.button("Button2")
col5, col6, col7, col8 = st.columns(4)
with col5:
    st.markdown(create_metric_card("New Leads", "320"), unsafe_allow_html=True)
with col6:
    st.markdown(create_metric_card("Churn Rate", "5%"), unsafe_allow_html=True)
with col7:
    st.markdown(create_metric_card("Website Visits", "12,000"), unsafe_allow_html=True)
with col8:
    st.markdown(create_metric_card("Conversion Rate", "8%"), unsafe_allow_html=True)

# ======================= FOOTER =======================
st.markdown("<h3 style='text-align:center; color:#9fb0cc; margin-top:50px;'> © 2026 Company Dashboard</h3>", unsafe_allow_html=True)
st.markdown("<h6 style='text-align:center; color:#9fb0cc; margin-top:50px;'> important links!</h6>", unsafe_allow_html=True)


st.markdown("""<h1 style='text-align:center; color:#9fb0cc; margin-top:50px; font-size:80px;>Heading 1</h1>
<h2 style='text-align:center; color:#9fb0cc; margin-top:50px;'>Heading 2</h2>
<h3 style='text-align:center; color:#9fb0cc; margin-top:0px;'>Heading 3</h3>
<h4 style='text-align:center; color:#9fb0cc; margin-top:50px;'>Heading 4</h4>
<h5 style='text-align:center; color:#9fb0cc; margin-top:50px;'>Heading 5</h5>
<h6 style='text-align:center; color:#9fb0cc; margin-top:50px;'>Heading 6</h6>""",unsafe_allow_html=True)


st.markdown("<p>This is<br>a paragraph<br>with line breaks.</p>",unsafe_allow_html=True)

st.markdown("""<pre>
            My Bonnie lies over the ocean.
            My Bonnie lies over the sea.
            My Bonnie lies over the ocean.
            Oh, bring back my Bonnie to me.
            </pre>""",unsafe_allow_html=True)

st.markdown("""<div style='background-color:black;color:white;padding:20px;'>
  <h2>London</h2>
  <p>London is the capital city of England. It is the most populous city in the United Kingdom, with a metropolitan area of over 13 million inhabitants.</p>
  <p>Standing on the River Thames, London has been a major settlement for two millennia, its history going back to its founding by the Romans, who named it Londinium.</p>
            <h1 style= 'background-color:White;color:blue; text-align: center;'> MACROECONOMIC DASHBOARD IS GREAT</h1>
</div>""",unsafe_allow_html=True)